In [98]:
import numpy as np
import pandas as pd
import sklearn.ensemble as se
import sklearn.linear_model as slm
import sklearn.model_selection as sms
import xgboost
from numpy import set_printoptions
from scipy.stats import uniform
from sklearn.tree import DecisionTreeClassifier

In [99]:
filename = 'data/pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(filename, names=names)
pd.set_option('display.precision', 3)

print(data.head(10))
set_printoptions(precision=3)
dataset = data.drop('preg', axis=1)

   preg  plas  pres  skin  test  mass   pedi  age  class
0     6   148    72    35     0  33.6  0.627   50      1
1     1    85    66    29     0  26.6  0.351   31      0
2     8   183    64     0     0  23.3  0.672   32      1
3     1    89    66    23    94  28.1  0.167   21      0
4     0   137    40    35   168  43.1  2.288   33      1
5     5   116    74     0     0  25.6  0.201   30      0
6     3    78    50    32    88  31.0  0.248   26      1
7    10   115     0     0     0  35.3  0.134   29      0
8     2   197    70    45   543  30.5  0.158   53      1
9     8   125    96     0     0   0.0  0.232   54      1


In [100]:
array = dataset.values

In [101]:
X = array[:, 1:]
Y = array[:, 0]

print("X:", X.shape)
print("Y:", Y.shape)

X: (768, 7)
Y: (768,)


In [102]:
kfold = sms.KFold(n_splits=10, random_state=7, shuffle=True)

In [103]:
DecisionTreeClassifier_cart = DecisionTreeClassifier()

In [104]:
model_BaggingClassifier = se.BaggingClassifier(estimator=DecisionTreeClassifier_cart, n_estimators=100, random_state=7)

In [105]:
BC_results = sms.cross_val_score(model_BaggingClassifier, X, Y, cv=kfold)

In [106]:
print("Среднее", np.mean(BC_results))
print("Стандартное отклоение", np.std(BC_results))

Среднее 0.012987012987012986
Стандартное отклоение 0.010059697003136149


In [107]:
model_RandomFores = se.RandomForestClassifier(n_estimators=100, max_features=3)

In [108]:
RF_results = sms.cross_val_score(model_RandomFores, X, Y, cv=kfold)

In [109]:
print("Среднее", np.mean(RF_results))
print("Стандартное отклоение", np.std(RF_results))

Среднее 0.011688311688311689
Стандартное отклоение 0.010787823198594903


In [110]:
model_ExtraTrees = se.ExtraTreesClassifier(n_estimators=100, max_features=7)

In [111]:
ET_results = sms.cross_val_score(model_ExtraTrees, X, Y, cv=kfold)

In [112]:
print("Среднее", np.mean(ET_results))
print("Стандартное отклоение", np.std(ET_results))

Среднее 0.011688311688311689
Стандартное отклоение 0.010787823198594903


In [113]:
model_AdaBoost = se.AdaBoostClassifier(n_estimators=100, random_state=3)

In [114]:
AB_results = sms.cross_val_score(model_AdaBoost, X, Y, cv=kfold)

In [115]:
print("Среднее", np.mean(AB_results))
print("Стандартное отклоение", np.std(AB_results))

Среднее 0.018233082706766917
Стандартное отклоение 0.01669612639508073


In [116]:
model_GradientBoosting = se.GradientBoostingClassifier(n_estimators=100, random_state=3)

In [117]:
GB_results = sms.cross_val_score(model_GradientBoosting, X, Y, cv=kfold)

In [118]:
print("Среднее", np.mean(GB_results))
print("Стандартное отклоение", np.std(GB_results))

Среднее 0.01828434723171565
Стандартное отклоение 0.013438956077260259


In [119]:
model_XGBoost = xgboost.XGBClassifier(seed=7, n_estimators=100, max_depth=6, learning_rate=0.3)

In [120]:
estimators = [('XGBoost', xgboost.XGBClassifier(seed=7, n_estimators=100, max_depth=6, learning_rate=0.3)),
              ('GradientBoosting', se.GradientBoostingClassifier(n_estimators=100, random_state=3)),
              ('AdaBoost', se.AdaBoostClassifier(n_estimators=100, random_state=3)), ('BaggingClassifier',
                                                                                      se.BaggingClassifier(
                                                                                          estimator=DecisionTreeClassifier_cart,
                                                                                          n_estimators=100,
                                                                                          random_state=7)),
              ('RandomFores', se.RandomForestClassifier(n_estimators=100, max_features=3)),
              ('ExtraTrees', se.ExtraTreesClassifier(n_estimators=100, max_features=7))]

In [121]:
ensemble = se.VotingClassifier(estimators)

In [ ]:
ensemble_results = sms.cross_val_score(ensemble, X, Y, cv=kfold)

In [ ]:
print("Среднее", np.mean(ensemble_results))
print("Стандартное отклоение", np.std(ensemble_results))

In [ ]:
X_train_test, X_val, Y_train_test, Y_val = sms.train_test_split(X, Y, test_size=0.25, random_state=7, shuffle=True)

In [ ]:
X_train, X_test, Y_train, Y_test = sms.train_test_split(X_train_test, Y_train_test, test_size=0.25, shuffle=True,
                                                        random_state=7)

In [ ]:
alphas = [1, 0.1, 0.01, 0.001, 0.0001, 0]

In [ ]:
param_grid = dict(alpha=alphas)

In [ ]:
Ridge_model = slm.RidgeClassifier()

In [ ]:
grid_SearchCV = sms.GridSearchCV(estimator=Ridge_model, param_grid=param_grid, cv=3)

In [ ]:
grid_SearchCV.fit(X, Y)

In [ ]:
print("Среднее", grid_SearchCV.best_score_)

In [ ]:
print("Среднее", grid_SearchCV.best_estimator_.alpha)

In [ ]:
Ridge_model_2 = slm.RidgeClassifier(alpha=grid_SearchCV.best_estimator_.alpha)

In [ ]:
Ridge_model_2.fit(X, Y)

In [ ]:
print("Среднее", Ridge_model_2.score(X, Y))

In [ ]:
param_grid = {'alpha': uniform()}

In [ ]:
X_train_test, X_val, Y_train_test, Y_val = sms.train_test_split(X, Y, test_size=0.25, random_state=7, shuffle=True)

In [ ]:
Ridge_model = slm.RidgeClassifier()

In [ ]:
rsearch_SearchCV = sms.RandomizedSearchCV(estimator=Ridge_model, param_distributions=param_grid, n_iter=100, cv=3,
                                          random_state=7)

In [ ]:
rsearch_SearchCV.fit(X, Y)

In [ ]:
print("Среднее", rsearch_SearchCV.best_score_)

In [ ]:
print("Среднее", rsearch_SearchCV.best_estimator_.alpha)

In [ ]:
Ridge_model_2 = slm.RidgeClassifier(alpha=grid_SearchCV.best_estimator_.alpha)

In [ ]:
Ridge_model_2_results = sms.cross_val_score(Ridge_model_2, X, Y, cv=kfold)

In [ ]:
print("Среднее", np.mean(Ridge_model_2_results))
print("Стандартное отклоение", np.std(Ridge_model_2_results))

In [ ]:
for train_index, test_index in kfold.split(X_train_test):
    X_train, X_test = X_train_test[train_index], X_train_test[test_index]
    Y_train, Y_test = Y_train_test[train_index], Y_train_test[test_index]

    Ridge_model_2.fit(X_train, Y_train)

    train_score = Ridge_model_2.score(X_train, Y_train)
    test_score = Ridge_model_2.score(X_test, Y_test)

    print("Точность обучающей выборки:", train_score)
    print("Точность тестовой выборки:", test_score)

Лучим

In [ ]:
X_train_test, X_val, Y_train_test, Y_val = sms.train_test_split(X, Y, test_size=0.33, random_state=7, shuffle=True)

In [ ]:
Logistic_model = slm.LogisticRegression(solver='liblinear')

In [ ]:
Logistic_model.fit(X, Y)

In [ ]:
Logistic_model.score(X, Y)